In [1]:
import numpy as np
import struct
from astropy.io import fits

In [2]:
class MURaM_snapshot:
    def __init__(self,sim_dir,sim_iter):
        #This reads in the new muram format (zxy) and transposes it to xyz
        self.sim_iter=sim_iter
        
        header = np.loadtxt(sim_dir+'Header.'+str(sim_iter).zfill(6))
                        
        self.nz = np.int32(header[0])
        self.nx = np.int32(header[1])
        self.ny = np.int32(header[2])   
        
        self.dz = np.float64(header[3])
        self.dx = np.float64(header[4])
        self.dy = np.float64(header[5]) 
        
        self.time = np.float64(header[6])
        self.dt = np.float64(header[7])
        self.maxva = np.float64(header[8])
        
        self.xax = np.arange(self.nx,dtype=np.float64)*self.dx
        self.yax = np.arange(self.ny,dtype=np.float64)*self.dy
        self.zax = np.arange(self.nz,dtype=np.float64)*self.dz
    
        self.tem = np.memmap(sim_dir + 'eosT.' +str(self.sim_iter).zfill(6),dtype=np.float32,mode='r',shape=(self.ny,self.nx,self.nz)).swapaxes(0,1)
        self.pre = np.memmap(sim_dir + 'eosP.' +str(self.sim_iter).zfill(6),dtype=np.float32,mode='r',shape=(self.ny,self.nx,self.nz)).swapaxes(0,1)
        self.rho = np.memmap(sim_dir + 'result_prim_0.' +str(self.sim_iter).zfill(6),dtype=np.float32,mode='r',shape=(self.ny,self.nx,self.nz)).swapaxes(0,1)
        self.vz = np.memmap(sim_dir + 'result_prim_1.' +str(self.sim_iter).zfill(6),dtype=np.float32,mode='r',shape=(self.ny,self.nx,self.nz)).swapaxes(0,1)
        self.vx = np.memmap(sim_dir + 'result_prim_2.' +str(self.sim_iter).zfill(6),dtype=np.float32,mode='r',shape=(self.ny,self.nx,self.nz)).swapaxes(0,1)
        self.vy = np.memmap(sim_dir + 'result_prim_3.' +str(self.sim_iter).zfill(6),dtype=np.float32,mode='r',shape=(self.ny,self.nx,self.nz)).swapaxes(0,1)
        self._4 = np.memmap(sim_dir + 'result_prim_4.' +str(self.sim_iter).zfill(6),dtype=np.float32,mode='r',shape=(self.ny,self.nx,self.nz)).swapaxes(0,1)
        self.bz = np.memmap(sim_dir + 'result_prim_5.' +str(self.sim_iter).zfill(6),dtype=np.float32,mode='r',shape=(self.ny,self.nx,self.nz)).swapaxes(0,1)
        self.bx = np.memmap(sim_dir + 'result_prim_6.' +str(self.sim_iter).zfill(6),dtype=np.float32,mode='r',shape=(self.ny,self.nx,self.nz)).swapaxes(0,1)
        self.by = np.memmap(sim_dir + 'result_prim_7.' +str(self.sim_iter).zfill(6),dtype=np.float32,mode='r',shape=(self.ny,self.nx,self.nz)).swapaxes(0,1)
    
    def write_spinor_style(self,out_dir):
        hdr = fits.Header()
        hdr['NAXIS1']=self.nx, ' length of data axis 1'
        hdr['NAXIS2']=self.nz, ' length of data axis 2'
        hdr['NAXIS3']=self.ny, ' length of data axis 3'
        hdr['T_X']=np.float32(self.nx*self.dx), ' Total x (vertical) size [cm]'
        hdr['T_Y']=np.float32(self.nz*self.dz), ' Total y (vertical) size [cm]'
        hdr['T_Z']=np.float32(self.ny*self.dy), ' Total z (vertical) size [cm]'
        hdr['TIME']=self.time, ' Time [sec]'
        hdr['DESC']='I like Turtles', ' Descriptive comment'
        
        ## Temperature
        fits.writeto(out_dir+"eosT."+str(self.sim_iter).zfill(6)+".fits", self.tem.swapaxes(1,2), hdr, overwrite = True)
        ## Pressure
        fits.writeto(out_dir+"eosP."+str(self.sim_iter).zfill(6)+".fits", self.pre.swapaxes(1,2), hdr, overwrite = True)
        ## Density
        fits.writeto(out_dir+"result_0."+str(self.sim_iter).zfill(6)+".fits", self.rho.swapaxes(1,2), hdr, overwrite = True)
        ## M_x
        fits.writeto(out_dir+"result_1."+str(self.sim_iter).zfill(6)+".fits", (self.vx*self.rho).swapaxes(1,2), hdr, overwrite = True)
        ## M_z - vertical
        fits.writeto(out_dir+"result_2."+str(self.sim_iter).zfill(6)+".fits", (self.vz*self.rho).swapaxes(1,2), hdr, overwrite = True)
        ## M_y
        fits.writeto(out_dir+"result_3."+str(self.sim_iter).zfill(6)+".fits", (self.vy*self.rho).swapaxes(1,2), hdr, overwrite = True)
        ## Don't need Energy
        #Etot =  (self.eps+0.5*self.rho*(self.vz*self.vz+self.vz*self.vz+self.vy*self.vy)+0.5*(self.bz*self.bz+self.bx*self.bx+self.by*self.by))
        fits.writeto(out_dir+"result_4."+str(self.sim_iter).zfill(6)+".fits",self._4.swapaxes(1,2), hdr, overwrite = True)
        ## B_x
        fits.writeto(out_dir+"result_5."+str(self.sim_iter).zfill(6)+".fits", self.bx.swapaxes(1,2), hdr, overwrite = True)
        ## B_z -vertical
        fits.writeto(out_dir+"result_6."+str(self.sim_iter).zfill(6)+".fits", self.bz.swapaxes(1,2), hdr, overwrite = True)
        ## B_y
        fits.writeto(out_dir+"result_7."+str(self.sim_iter).zfill(6)+".fits", self.by.swapaxes(1,2), hdr, overwrite = True)
        

In [3]:
sim_dir = '/data/bhatia/jonas/G_100/3D/'#'/ptmp/damp/jonas_runs/100G_v2/3D/'
out_dir = '/data/slam/sinjan/tanay_cubes/100G/'
snapshot = MURaM_snapshot(sim_dir,'059932')
snapshot.write_spinor_style(out_dir)

In [3]:
sim_dir = '/data/bhatia/jonas/G_30/3D/'
out_dir = '/data/slam/sinjan/tanay_cubes/30G/'
sim_list = ['079245','081486','083742','086038','088303','090538','092779','095011','097188','099413','101564','103737']
for sim in sim_list:
    snapshot = MURaM_snapshot(sim_dir,sim)
    snapshot.write_spinor_style(out_dir)

In [3]:
sim_dir = '/data/bhatia/jonas/G_100/3D/'
out_dir = '/data/slam/sinjan/tanay_cubes/100G/'
sim_list = ['062302','064671','067043','069447','071858','074219','076639','079042','081434','083791','086170','088576']
for sim in sim_list:
    snapshot = MURaM_snapshot(sim_dir,sim)
    snapshot.write_spinor_style(out_dir)

In [3]:
sim_dir = '/data/bhatia/jonas/G_200/3D/'
out_dir = '/data/slam/sinjan/tanay_cubes/200G/'
sim_list = ['459704','462122','464531','466962','469389','471811','474220','476643','479085','481513','483915','486360']
for sim in sim_list:
    snapshot = MURaM_snapshot(sim_dir,sim)
    snapshot.write_spinor_style(out_dir)

In [4]:
#missed one
sim_dir = '/data/bhatia/jonas/G_200/3D/'
out_dir = '/data/slam/sinjan/tanay_cubes/200G/'
sim = '469389'
snapshot = MURaM_snapshot(sim_dir,sim)
snapshot.write_spinor_style(out_dir)

In [4]:
bz = fits.getdata('/data/slam/sinjan/tanay_cubes/100G/result_6.059932.fits')

In [5]:
np.mean(bz)*np.sqrt(4*np.pi)

100.00024057678782

In [6]:
bz = fits.getdata('/data/slam/sinjan/tanay_cubes/100G/result_5.059932.fits')
print(np.mean(bz)*np.sqrt(4*np.pi))

-8.40634298644435


In [7]:
bz = fits.getdata('/data/slam/sinjan/tanay_cubes/100G/result_7.059932.fits')
print(np.mean(bz)*np.sqrt(4*np.pi))

4.949662774818116


In [19]:
sim_dir = '/data/sun/przybylski/tino_restart_forjonas_10hr/100G_restart/100G_v2/3D/'#'/ptmp/damp/jonas_runs/100G_v2/3D/'
out_dir = '/data/slam/sinjan/tino_restart/100G/'
snapshot = MURaM_snapshot(sim_dir,202000)
snapshot.write_spinor_style(out_dir)

In [27]:
sim_list = [203000,204000,205000,206000,207000,208000,209000,210000,212000,213000,214000,215000,216000,217000,218000,219000,220000]
for sim in sim_list:
    snapshot = MURaM_snapshot(sim_dir,sim)
    snapshot.write_spinor_style(out_dir)

In [24]:
bz = fits.getdata('/data/slam/sinjan/tino_restart/100G/result_6.202000.fits')

In [26]:
np.mean(bz)*np.sqrt(4*np.pi)

99.99997012179347

In [28]:
bz = fits.getdata('/data/slam/sinjan/tino_restart/100G/result_6.220000.fits')

In [29]:
np.mean(bz)*np.sqrt(4*np.pi)

99.99999040591804

In [ ]:
#120G restart

In [3]:
sim_dir = '/data/sun/przybylski/tino_restart_forjonas_10hr/120G_restart/120G/3D/'#'/ptmp/damp/jonas_runs/100G_v2/3D/'
out_dir = '/data/slam/sinjan/tino_restart/120G/'
# snapshot = MURaM_snapshot(sim_dir,202000)
# snapshot.write_spinor_style(out_dir)

In [4]:
sim_list = [202000,203000,204000,205000,206000,207000,208000,209000,210000,212000,213000,214000,215000,216000,217000,218000,219000,220000]
for sim in sim_list:
    snapshot = MURaM_snapshot(sim_dir,sim)
    snapshot.write_spinor_style(out_dir)

In [6]:
bz = fits.getdata('/data/slam/sinjan/tino_restart/120G/result_6.210000.fits')
np.mean(bz)*np.sqrt(4*np.pi)

120.00000200985137

In [8]:
bx = fits.getdata('/data/slam/sinjan/tino_restart/120G/result_5.210000.fits')
np.mean(bx)*np.sqrt(4*np.pi)

5.139954908835621

In [13]:
bx = fits.getdata('/data/sunrise/mhd/MURaM/MySimulations/MURaM_RUN_1/20161029.100G.ngrey.288x100x288/result_5.160000.fits')
np.mean(bx)*np.sqrt(4*np.pi)

1.3649787498864108